## Portfolios and Portfolio Groups

[Portfolios in LUSID](https://support.lusid.com/portfolios) are the main containers for transactions and holdings, and come in multiple forms based on hierarchy and use case. The basic portfolio types are :
- Transaction Portfolios
- Reference Portfolios
- Derived Portfolios

These portfolios can then be [grouped](https://support.lusid.com/portfolios#portfolio-groups), where portfolio operations on the group are applied to an aggregated set of portfolios in the hierarchy.

This example will show how to :
1. Create Transactions Portfolios
2. Create Derived Portfolios
3. (add/get/delete portfolio properties and details)
4. Create portfolio Groups
5. Add portfolios to a Group
6. Add a group to a group (creation of subgroups)
7. Modify groups
8. Remove portfolios and subgroups from groups
9. Delete portfolios
10. Delete groups
11. Get all the commands applied to a group or portfolio
12. List all the groups and portfolios in a scope

Initialise our environment and connect to LUSID

In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
from msrest.authentication import BasicTokenAuthentication

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print('LUSID Environment Initialised')
print('LUSID version : ', client.api_version)

LUSID Environment Initialised
LUSID version :  0.9.116


First we start by loading some default portfolios, we will put them in two different scopes to demonstrate the separation between scopes and the usage of different scopes within portfolio groups.

- Note: In order to demonstrate the deletion more clearly at then end of this example, run the portfolio creation more than once, and the derived portfolio creation 2-3 times as well, this will create a population of unnecessary portfolios that can be safely removed without affecting the portfolios of interest. It is not necessary to 

### Initial Setup

In [2]:
# Use pandas to read a csv containing the information needed to create some portfolios
df = pd.read_csv("data/dummy_portfolios_names.csv")

# We can also save the headers to a separate variable for easier use
headers = df.columns
print("number of columns : ", headers.size)
print("column names : ", headers)

df.head()

number of columns :  2
column names :  Index(['display_name', 'base_currency'], dtype='object')


,display_name,base_currency
0,portfolio-A,GBP
1,portfolio-B,GBP
2,portfolio-C,GBP
3,portfolio-D,GBP
4,portfolio-E,GBP


In [3]:
# Create a transaction Portfolio - extract the scopes from the csv
scope1 = 'portfolio_demo_abc'
scope2 = 'portfolio_demo_xyz'
print("Scope1 : ", scope1)
print("Scope2 : ", scope2)

Scope1 :  portfolio_demo_abc
Scope2 :  portfolio_demo_xyz


The cell below contains a number of helper functions used throughout this notebook - these are not necessary for any of the methods demonstrated, but simplify the readability and usability of this use-case example

In [4]:
# The following function creates a random alphanumeric code of 4 characters that can be appended to Ids
# and Names to ensure they remain unique throughout multiple runs of this example
def GetGuid():
    return str(uuid.uuid4())[:4]

# The following function will create portfolios from our dataframe of csv data
def CreatePortfolioFromDataframe(df):
    effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)
    list_results = []
    for index, row in df.iterrows():
        print("Creating portfolio ", index)
        guid = GetGuid()
        # Select the scope for each portfolio based on currency, for demonstration purposes we will
        # split GBP based portfolios into scope 1 and USD based portfolios into scope 2
        if row['base_currency'] == 'USD':
            portfolio_scope = scope2
        else :
            portfolio_scope = scope1
        # Next we create a request for the portfolio
        request = models.CreateTransactionPortfolioRequest(
            code=row['display_name']+'-'+guid,
            display_name=row['display_name'],
            base_currency=row['base_currency'],
            created=effective_date,
            description=None,
            corporate_action_source_id=None,
            accounting_method=None,
            sub_holding_keys=None,
            properties=None)
        # And finally we can upsert the portfolio creation request to LUSID
        result = client.create_portfolio(
            scope=portfolio_scope,
            create_request=request)
        # Save the portfolio to a list for easy access later on
        list_results.append(result)
        print('...')
    # Return the list of portfolios that were upserted in LUSID
    return list_results

# The following function extracts resource ids from a list of portfolios
def GetResourceIdsFromResponse(responses):
    resource_id_list = []
    for portfolio in responses:
        resource_id_list.append(portfolio.id)
    return resource_id_list





We begin by upserting the csv data into LUSID in the form of new transaction portfolios

In [5]:
created_portfolios = CreatePortfolioFromDataframe(df)

# we can now prettyprint all our portfolio responses to see the info on each new portfolio
for portfolio in created_portfolios:
    prettyprint.portfolio_response(portfolio)

Creating portfolio  0
...
Creating portfolio  1
...
Creating portfolio  2
...
Creating portfolio  3
...
Creating portfolio  4
...
Creating portfolio  5
...
Creating portfolio  6
...
Creating portfolio  7
...
Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-A-253c
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:51.566017+00:00

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-B-8c1e
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:53.405680+00:00

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-C-913d
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:53.681816+00:00

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-D-129a
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:53.933950+00:00

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-E-9e3a
Portfolio Effective From:

We then create one [derived portfolio](https://support.lusid.com/portfolios#derived-portfolios) from the first transaction portfolio created.

In [6]:
# Create derived transaction portfolio from the first portfolio in the list
guid = GetGuid()
derived_portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
    display_name="derived-"+created_portfolios[0].display_name, # this uses the portfolio name
    code="derived-"+created_portfolios[0].display_name+'-'+guid,
    parent_portfolio_id=models.ResourceId(
        scope=created_portfolios[0].id.scope,
        code=created_portfolios[0].id.code),
    description="derived_portfolio_description",
    created=created_portfolios[0].created,
    corporate_action_source_id=None,
    accounting_method=None,
    sub_holding_keys=None)

derived_portfolio = client.create_derived_portfolio(
    scope=scope1,
    portfolio=derived_portfolio_request)

prettyprint.portfolio_response(derived_portfolio)

Derived Portfolio Created
Scope: portfolio_demo_abc
Code: derived-portfolio-A-8665
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:55.293452+00:00

   Parent Portfolio Details
   Code: portfolio-A-253c



A derived portfolio will automatically inherit all of the parent portfolio's details if they are not specified. These details can be modified and/or deleted. 
Modifying the details on a derived portfolio will not affect the parent portfolio. This is demonstrated in the following cells

In [7]:
# First we will get the details for our new derived portfolio
# and check they match with the parent portfolio
derived_details = client.get_details(
    scope=scope1,
    code=derived_portfolio.id.code,
    effective_at=None,
    as_at=None)
print("Derived Portfolio Details : ")
prettyprint.portfolio_details_response(derived_details)

print("Parent Portfolio Details : ")
parent_details = client.get_details(
    scope=scope1,
    code=created_portfolios[0].id.code,
    effective_at=None,
    as_at=None)
prettyprint.portfolio_details_response(parent_details)

Derived Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_abc
Detail Origin Portfolio Code: portfolio-A-253c
Base Currency: GBP


Parent Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_abc
Detail Origin Portfolio Code: portfolio-A-253c
Base Currency: GBP




Now we can modify the derived portfolio currency so it no longer matches the parent portfolio

In [8]:
request = models.CreatePortfolioDetails(
    base_currency="USD",
    corporate_action_source_id=None)
response = client.upsert_portfolio_details(
    scope=scope1,
    code=derived_portfolio.id.code,
    details=request,
    effective_at=None)
print("Derived Portfolio Details : ")
prettyprint.portfolio_details_response(response)

print("Parent Portfolio Details : ")
parent_details = client.get_details(
    scope=scope1,
    code=created_portfolios[0].id.code,
    effective_at=None,
    as_at=None)
prettyprint.portfolio_details_response(parent_details)

Derived Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_abc
Detail Origin Portfolio Code: derived-portfolio-A-8665
Base Currency: USD


Parent Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_abc
Detail Origin Portfolio Code: portfolio-A-253c
Base Currency: GBP




Now we can delete portfolio details. Note how after deleting the details on the derived portfolio, the response returns the scope and code of the parent portfolio, since `get_details` only returns the id of the portfolio from which the details originate - not the portfolio being queried.

In [9]:
response = client.delete_derived_portfolio_details(
    scope=scope1,
    code=derived_portfolio.id.code,
    effective_at=None,
    as_at=None)
derived_details = client.get_details(
    scope=scope1,
    code=derived_portfolio.id.code,
    effective_at=None,
    as_at=None)

print("Derived Portfolio Details : ")
prettyprint.portfolio_details_response(derived_details) 

print("Parent Portfolio Details : ")
parent_details = client.get_details(
    scope=scope1,
    code=created_portfolios[0].id.code)
prettyprint.portfolio_details_response(parent_details)

Derived Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_abc
Detail Origin Portfolio Code: portfolio-A-253c
Base Currency: GBP


Parent Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_abc
Detail Origin Portfolio Code: portfolio-A-253c
Base Currency: GBP




Next we can create some portfolio groups [(more details here)](https://support.lusid.com/portfolios#portfolio-groups). We will then demonstrate the process of adding portfolios to a group, creating subgroups, and modifying group information.

To begin with, let's create a group in our main scope that contains all the GBP parent portfolios.

Note that the second group is in a different scope to the first, it is entirely possible to combine portfolios and groups in different scopes, keep in mind that group-wide operations will still only apply to the specified scope, and will not affect portfolios/subgroups in different scopes.

In [10]:
# Create a portfolio group -- create two
# -- The first group we will create with all the portfolios in the first scope
# -- we can use a helper function to obtain a list of resource ids for this
scope_1_portfolios = [(x) for x in created_portfolios if x.id.scope == scope1]
scope_1_resource_ids = GetResourceIdsFromResponse(scope_1_portfolios)

# Create group 1 with the portfolios of scope 1 in it
guid = GetGuid()
group_request1 = models.CreatePortfolioGroupRequest(
    id="portfolio_group-{0}".format(guid),
    display_name="portfolio_group1_name",
    values=scope_1_resource_ids,
    sub_groups=None,
    description=None)

portfolio_group1 = client.create_portfolio_group(scope1, group_request1)
prettyprint.portfolio_group_response(portfolio_group1, 'created')


Portfolio Group Created
Name: portfolio_group1_name
Scope: portfolio_demo_abc
Code: portfolio_group-6c02
Portfolios Inside Group: 
portfolio-A-253c
portfolio-B-8c1e
portfolio-C-913d
portfolio-D-129a
portfolio-E-9e3a
portfolio-F-4e90




So we have one group pre-populated with the portfolios in the first scope `portfolio_demo_main`. Since the derived portfolio hasn't been included in the first group, we can add it in now to Group 1.

In [11]:
# adding the derived portfolio to group 1 :
client.add_portfolio_to_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    portfolio_id=derived_portfolio.id)

We can now compare two methods of obtaining groups from LUSID. The first is `get portfolio group` while the second is `get portfolio group expansion`. The difference here is that the first method will return only information about the specified group, including the ids of the portfolios and subgroups it contains, but not any further details of its portfolios and subgroups. The second method will not only return this basic information, but also the full details of each portfolio in the group and each portfolio in every subgroup as well. 


First we compare group 1 at the time of creation and at the most up-to-date time below, so we can see the changes that have been made to it since we created it using `get portfolio group`. The only expected difference right now will be the added derived portfolio. We will demonstrate the expanded method further on.

In [12]:
# Get portfolio group now
updated_group1 = client.get_portfolio_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    as_at=datetime.utcnow().replace(tzinfo=pytz.utc))

# Get portfolio group 1 prior to the update
original_group1 = client.get_portfolio_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    as_at=portfolio_group1.version.as_at_date)

print('Group 1 at time of creation')
prettyprint.get_portfolio_group_response(original_group1)
print('')
print('Group 1 at latest time')
prettyprint.get_portfolio_group_response(updated_group1)

Group 1 at time of creation
Portfolio Group: 
Name: portfolio_group1_name
Scope: portfolio_demo_abc
Code: portfolio_group-6c02
Portfolios Inside Group: 
portfolio-A-253c
portfolio-B-8c1e
portfolio-C-913d
portfolio-D-129a
portfolio-E-9e3a
portfolio-F-4e90
Subgroups Inside Group: 



Group 1 at latest time
Portfolio Group: 
Name: portfolio_group1_name
Scope: portfolio_demo_abc
Code: portfolio_group-6c02
Portfolios Inside Group: 
portfolio-A-253c
portfolio-B-8c1e
portfolio-C-913d
portfolio-D-129a
portfolio-E-9e3a
portfolio-F-4e90
derived-portfolio-A-8665
Subgroups Inside Group: 




This leaves the two portfolios in Scope 2. Since they are in another scope and based in a different currency, it makes sense to place them in a separate group to the first one. This time we create an empty group, then populate it separately. We follow this process as opposed to creating a pre-populated group (such as group 1 above) to demonstrate how empty groups can be created and how to upsert multiple potfolios to a group.

In [13]:
# Create group 2 with no portfolios
guid = GetGuid()
group_request2 = models.CreatePortfolioGroupRequest(
    id="portfolio_group-{0}".format(guid),
    display_name="portfolio_group2_name",
    values=None,
    sub_groups=None,
    description=None)

portfolio_group2 = client.create_portfolio_group(scope2, group_request2)
prettyprint.portfolio_group_response(portfolio_group2, 'created')

# Add portfolios to groups
# Once again we get a list of resource ids for the portfolios in scope 2 this time
scope_2_portfolios = [(x) for x in created_portfolios if x.id.scope == scope2]
scope_2_resource_ids = GetResourceIdsFromResponse(scope_2_portfolios)

# Then to add multiple portfolios to group 2, we have to iterate over the list of ids, since they have to
# be added one by one.
for resource_id in scope_2_resource_ids:
    client.add_portfolio_to_group(
        scope=portfolio_group2.id.scope,
        code=portfolio_group2.id.code,
        portfolio_id=resource_id)

Portfolio Group Created
Name: portfolio_group2_name
Scope: portfolio_demo_xyz
Code: portfolio_group-1b1e
Portfolios Inside Group: 




We can also combine groups into each other to create sub-groups. We will add group 2 as a subgroup in group 1.

In [14]:
# Add group to group
client.add_sub_group_to_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    portfolio_group_id=portfolio_group2.id)

Since group 2 is now a subgroup, we can update its display name to reflect this

In [15]:
# Update group request
group_update_request = models.UpdatePortfolioGroupRequest(
    display_name="Group1_Subgroup",
    description=None)

# Update group
client.update_portfolio_group(
    scope=portfolio_group2.id.scope,
    code=portfolio_group2.id.code,
    request=group_update_request)

We can now compare the `get _portfolio_group` method with getting the full expansion of a group. You will see that on top of a list of portfolios and subgroups, the full expansion also returns all the information for the portfolios similar to running the `get_portfolio` method on each portfolio in the group and groubgroup.

In [16]:
# Get the basic group information for group 1 again:
updated_group1 = client.get_portfolio_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    as_at=datetime.utcnow().replace(tzinfo=pytz.utc))

print('Group 1 at latest time with basic GET method : ')
prettyprint.get_portfolio_group_response(updated_group1)


# Get full expansion of portfolio group 1:
expansion = client.get_portfolio_group_expansion(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    effective_at=None,
    as_at=None,
    property_filter=None)

prettyprint.expanded_portfolio_group_response(expansion)

Group 1 at latest time with basic GET method : 
Portfolio Group: 
Name: portfolio_group1_name
Scope: portfolio_demo_abc
Code: portfolio_group-6c02
Portfolios Inside Group: 
portfolio-A-253c
portfolio-B-8c1e
portfolio-C-913d
portfolio-D-129a
portfolio-E-9e3a
portfolio-F-4e90
derived-portfolio-A-8665
Subgroups Inside Group: 
portfolio_group-1b1e


Portfolio Group Full Details : 
Scope: portfolio_demo_abc
Code: portfolio_group-6c02
Portfolios Inside Group: 
Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-A-253c
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:55.293452+00:00

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-B-8c1e
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:53.405680+00:00

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-C-913d
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:53.681816+00:00

Portfolio Create

Suppose we wish to revert these changes, we have to explicitly delete each component step by step. Our Subgroup contains two portfolios, we're going to remove these from that group, remove the subgroup from the main group, and then delete it as it is no longer needed. If a subgroup is empty, it is not necessary to remove it before deleting it completely, however for example purposes we will follow each step.

Note : it is possible to delete a populated group. This will not delete the portfolios, they will simply be removed from the group hierarchy.

In [17]:
# 1. Removal of portfolios from a group has to be done one portfolio at a time,
# thus we can loop through the list of portfolios in scope 2 since they are all contained in the subgroup

# first we print out the details of group 2
original_group2 = client.get_portfolio_group(
    scope=portfolio_group2.id.scope,
    code=portfolio_group2.id.code,
    as_at=None)

prettyprint.get_portfolio_group_response(original_group2)
print('')

# then we proceed with removing each portfolio in the group, you will see each printed response will have
# one fewer portfolio ids attached to the group.
for portfolio_id in scope_2_resource_ids:
    portfolio_removal = client.delete_portfolio_from_group(
        scope=portfolio_group2.id.scope,
        code=portfolio_group2.id.code,
        portfolio_scope=portfolio_id.scope,
        portfolio_code=portfolio_id.code)
    prettyprint.portfolio_group_response(portfolio_removal, 'none')

Portfolio Group: 
Name: Group1_Subgroup
Scope: portfolio_demo_xyz
Code: portfolio_group-1b1e
Portfolios Inside Group: 
portfolio-X-4400
portfolio-Y-3c97
Subgroups Inside Group: 



Portfolio Group
Name: Group1_Subgroup
Scope: portfolio_demo_xyz
Code: portfolio_group-1b1e
Portfolios Inside Group: 
portfolio-Y-3c97


Portfolio Group
Name: Group1_Subgroup
Scope: portfolio_demo_xyz
Code: portfolio_group-1b1e
Portfolios Inside Group: 




In [18]:
# 2. Remove group from group
group_removal = client.delete_sub_group_from_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    subgroup_scope=portfolio_group2.id.scope,
    subgroup_code=portfolio_group2.id.code)

prettyprint.portfolio_group_response(group_removal, 'none')
print('Remaining Sub Groups : ', group_removal.sub_groups)

Portfolio Group
Name: portfolio_group1_name
Scope: portfolio_demo_abc
Code: portfolio_group-6c02
Portfolios Inside Group: 
portfolio-A-253c
portfolio-B-8c1e
portfolio-C-913d
portfolio-D-129a
portfolio-E-9e3a
portfolio-F-4e90
derived-portfolio-A-8665


Remaining Sub Groups :  []


In [19]:
# 3. Delete group
group_deletion = client.delete_portfolio_group(
    scope=portfolio_group2.id.scope,
    code=portfolio_group2.id.code)

We can now check what operations have been applied to each of our groups, including an already deleted group, by using the `get portfolio group commands` method, as below.

In [20]:
# Get commands
group1_commands = client.get_portfolio_group_commands(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    from_as_at=None,
    to_as_at=None,
    sort_by=None,
    start=None,
    limit=None,
    filter=None)

group2_commands = client.get_portfolio_group_commands(
    scope=portfolio_group2.id.scope,
    code=portfolio_group2.id.code,
    from_as_at=None,
    to_as_at=None,
    sort_by=None,
    start=None,
    limit=None,
    filter=None)

prettyprint.group_commands(group1_commands, portfolio_group1.display_name)

prettyprint.group_commands(group2_commands, portfolio_group2.display_name)

Commands Applied To Group portfolio_group1_name
Number of commands : 4
Description : Create portfolio group
At Time : 2019-02-22T16:37:56.9048880+00:00


Description : Add portfolio to group
At Time : 2019-02-22T16:37:57.2266840+00:00


Description : Add subgroup to group
At Time : 2019-02-22T16:37:58.6841460+00:00


Description : Delete subgroup from group
At Time : 2019-02-22T16:38:00.2187940+00:00


Commands Applied To Group portfolio_group2_name
Number of commands : 7
Description : Create portfolio group
At Time : 2019-02-22T16:37:57.7635160+00:00


Description : Add portfolio to group
At Time : 2019-02-22T16:37:58.0372060+00:00


Description : Add portfolio to group
At Time : 2019-02-22T16:37:58.3258600+00:00


Description : Update portfolio group
At Time : 2019-02-22T16:37:58.9356050+00:00


Description : Delete portfolio from group
At Time : 2019-02-22T16:37:59.6510680+00:00


Description : Delete portfolio from group
At Time : 2019-02-22T16:37:59.9372530+00:00


Description : D

We can now verify that the group has indeed been deleted from our scope by getting all groups in each of our scopes, the second of which should be empty now.

In [21]:
# List groups in scope - show one scope with no groups
scope1_groups = client.list_portfolio_groups(scope=scope1)
scope2_groups = client.list_portfolio_groups(scope=scope2)
print("Groups in scope 1 : ")
for group in scope1_groups.values:
    print('    Scope : ', group.id.scope+', Code : ', group.id.code)

print("Groups in scope 2 : ")
for group in scope2_groups.values:  # This one should be empty and not output anything
    print('    Scope : ', group.id.scope+', Code : ', group.id.code)

Groups in scope 1 : 
    Scope :  portfolio_demo_abc, Code :  portfolio_group-6c02
Groups in scope 2 : 


In case any extra groups are hanging around, the next cell will remove all groups. This will also remove Group 1 in Scope 1, which is still populated, showing how the portfolios remain in the scope and ungrouped following this operation.

In [22]:
# create a list of all groups in both scopes
groups_to_delete = []
for group in scope1_groups.values:
        groups_to_delete.append(group.id)
for group in scope2_groups.values:
        groups_to_delete.append(group.id)

# loop through the list of groups and delete each one, one by one
for group in groups_to_delete:
    client.delete_portfolio_group(
        scope = group.scope,
        code = group.code)
    
# List groups in scope - both will now be empty
scope1_groups = client.list_portfolio_groups(scope=scope1)
scope2_groups = client.list_portfolio_groups(scope=scope2)
print("Groups in scope 1 : ")
for group in scope1_groups.values:
    print('    Scope : ', group.id.scope+', Code : ', group.id.code)

print("Groups in scope 2 : ")
for group in scope2_groups.values:  # This one should be empty and not output anything
    print('    Scope : ', group.id.scope+', Code : ', group.id.code)


Groups in scope 1 : 
Groups in scope 2 : 


If we list all the portfolios in our scopes we will see that they are all still there. There may be extra portfolios not created in this example, if this is the case, we can delete all these extra portfolios and keep only those we are interested in. Note that scope 2 is now completely empty of portfolios.

In [23]:
scope1_portfolios_list_response = client.list_portfolios_for_scope(
    scope=scope1,
    effective_at=None,
    as_at=None,
    sort_by='-as_at',
    start=None,
    limit=None,
    filter=None)
scope2_portfolios_list_response = client.list_portfolios_for_scope(
    scope=scope2,
    effective_at=None,
    as_at=None,
    sort_by=None,
    start=None,
    limit=None,
    filter=None)

print('Portfolios remaining in scope ', scope1, ' :\n')
for portfolio in scope1_portfolios_list_response.values:
    prettyprint.portfolio_response(portfolio)

print('Portfolios remaining in scope ', scope2, ' :\n')
for portfolio in scope2_portfolios_list_response.values:
    prettyprint.portfolio_response(portfolio)

Portfolios remaining in scope  portfolio_demo_abc  :

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-C-913d
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:53.681816+00:00

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-A-253c
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:55.293452+00:00

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-A-9ebe
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:05.203006+00:00

Derived Portfolio Created
Scope: portfolio_demo_abc
Code: derived-portfolio-A-5a98
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:06.821586+00:00

   Parent Portfolio Details
   Code: portfolio-A-9ebe

Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-D-129a
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:53.933950+00:00



The cells below allow us to delete the extra portfolios in the scope. Keep in mind, derived portfolios must be deleted first, as if a parent portfolio still contains a child, the deletion will fail.

In [24]:
# Helper functions to filter out unwanted portfolios

def extractIds(portfolioList):
    parent_ids = []
    derived_ids = []
    for folio in portfolioList:
        if folio.is_derived:
            derived_ids.append((folio.id.scope, folio.id.code))
        else:
            parent_ids.append((folio.id.scope, folio.id.code))
    return (parent_ids, derived_ids)

def returnNotMatches(a, b):
    if type(b) is not tuple:
        return [x for x in a if x not in b]
    else :
        return [x for x in a if x not in [b]]


Now we select the parent/derived portfolios we wish to keep or delete. Currently `portfolios_to_keep` is hardcoded to keep the derived portfolio that we created at the beginning of this example, and its parent portfolio.

In [25]:
# first combine all the portfolios in our two scopes
all_portfolios_in_scopes = extractIds(scope1_portfolios_list_response.values
                                      + scope2_portfolios_list_response.values)
parent_portfolios = all_portfolios_in_scopes[0]
derived_portfolios = all_portfolios_in_scopes[1]

# then combine the portfolios we're interested in keeping into a list 
# - just the first portfolio we created and its derived portfolio
portfolios_to_keep = [(derived_portfolio.parent_portfolio_id.scope, derived_portfolio.parent_portfolio_id.code),
                     (derived_portfolio.id.scope, derived_portfolio.id.code)]
parents_to_keep = portfolios_to_keep[0]
derived_to_keep = portfolios_to_keep[1]

# we can now exclude portfolios to keep from the full list
parents_to_delete = returnNotMatches(parent_portfolios, parents_to_keep)
derived_to_delete = returnNotMatches(derived_portfolios, derived_to_keep)
print("Keeping Parents : ", parents_to_keep)
print("Deleting Parents : ", parents_to_delete)
print("")
print("Keeping Derived : ", derived_to_keep)
print("Deleting Derived : ", derived_to_delete)

Keeping Parents :  ('portfolio_demo_abc', 'portfolio-A-253c')
Deleting Parents :  [('portfolio_demo_abc', 'portfolio-C-913d'), ('portfolio_demo_abc', 'portfolio-A-9ebe'), ('portfolio_demo_abc', 'portfolio-D-129a'), ('portfolio_demo_abc', 'portfolio-F-4e90'), ('portfolio_demo_abc', 'portfolio-B-8c1e'), ('portfolio_demo_abc', 'portfolio-E-9e3a'), ('portfolio_demo_xyz', 'portfolio-X-4400'), ('portfolio_demo_xyz', 'portfolio-Y-3c97')]

Keeping Derived :  ('portfolio_demo_abc', 'derived-portfolio-A-8665')
Deleting Derived :  [('portfolio_demo_abc', 'derived-portfolio-A-5a98')]


With this filtering done, we can proceed with deleting unwanted portfolios. Deletion is done one portfolio at a time, therefore we must loop through the list of portfolios to delete. Derived portfolios are deleted first, followed by a separate loop to delete parent (top level - they may never have had derived portfolios attached) portfolios.

In [26]:
# Next we delete the derived portfolios first
print("Deleting Derived Portfolios ... ")
for portfolio in derived_to_delete:
    response = client.delete_portfolio(
        scope=portfolio[0],
        code=portfolio[1],
        effective_at=None)
    print('Portfolio deleted : ', portfolio[0], portfolio[1])
    print('At time : ', str(response.as_at))
print('\n')
print("Deleting remaining Portfolios ... ")
# Then we proceed with deleting the remaining parent portfolios
for portfolio in parents_to_delete:
    response = client.delete_portfolio(
        scope=portfolio[0],
        code=portfolio[1],
        effective_at=None)
    print('Portfolio deleted : ', portfolio[0], portfolio[1])
    print('At time : ', str(response.as_at))

Deleting Derived Portfolios ... 
Portfolio deleted :  portfolio_demo_abc derived-portfolio-A-5a98
At time :  2019-02-22 16:38:02.990861+00:00


Deleting remaining Portfolios ... 
Portfolio deleted :  portfolio_demo_abc portfolio-C-913d
At time :  2019-02-22 16:38:03.366530+00:00
Portfolio deleted :  portfolio_demo_abc portfolio-A-9ebe
At time :  2019-02-22 16:38:03.636770+00:00
Portfolio deleted :  portfolio_demo_abc portfolio-D-129a
At time :  2019-02-22 16:38:03.874603+00:00
Portfolio deleted :  portfolio_demo_abc portfolio-F-4e90
At time :  2019-02-22 16:38:04.154174+00:00
Portfolio deleted :  portfolio_demo_abc portfolio-B-8c1e
At time :  2019-02-22 16:38:04.427678+00:00
Portfolio deleted :  portfolio_demo_abc portfolio-E-9e3a
At time :  2019-02-22 16:38:04.786114+00:00
Portfolio deleted :  portfolio_demo_xyz portfolio-X-4400
At time :  2019-02-22 16:38:05.049275+00:00
Portfolio deleted :  portfolio_demo_xyz portfolio-Y-3c97
At time :  2019-02-22 16:38:05.315028+00:00


If we now re-list the portfolios in our scopes, we will see that only our two original portfolios remain, and one derived portfolio.

In [27]:
# We can re-run the list portfolios command to check what portfolios remain in each scope
scope1_portfolios_list_response = client.list_portfolios_for_scope(
    scope=scope1,
    effective_at=None,
    as_at=None,
    sort_by=None,
    start=None,
    limit=None,
    filter=None)

scope2_portfolios_list_response = client.list_portfolios_for_scope(
    scope=scope2,
    effective_at=None,
    as_at=None,
    sort_by=None,
    start=None,
    limit=None,
    filter=None)

print('\033[91m' + 'Portfolios remaining in scope: ' + '\033[0m', scope1, ':')
for portfolio in scope1_portfolios_list_response.values:
    prettyprint.portfolio_response(portfolio)

print('\033[91m' + 'Portfolios remaining in scope: ' + '\033[0m', scope2, ':')
for portfolio in scope2_portfolios_list_response.values:
    prettyprint.portfolio_response(portfolio)
    print('\n')

Portfolios remaining in scope:  portfolio_demo_abc :
Portfolio Created
Scope: portfolio_demo_abc
Code: portfolio-A-253c
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:55.293452+00:00

Derived Portfolio Created
Scope: portfolio_demo_abc
Code: derived-portfolio-A-8665
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-22 16:37:56.314887+00:00

   Parent Portfolio Details
   Code: portfolio-A-253c

Portfolios remaining in scope:  portfolio_demo_xyz :
